<a href="https://colab.research.google.com/github/NiveditJain/dataProcessingUsingSQLlite/blob/master/CarPoolPeopleDataSetGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing all the required header file
import pandas as pd 
import sqlite3
import os
import gc
import csv
from google.colab import drive

In [9]:
# connecting data file from google
drive.mount('/content/gdrive')
# location of data
path='/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/edges.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
outputPath='/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'
inputPath='/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/'

In [0]:
# coverting the required part of data to csv file and reallocating memory
def trimCSV(starting,Termination,name):
  if(os.path.exists('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')==1):
                    os.remove('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')
  df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Twitter-dataset/data/edges.csv',names=['person','following'])
  if(not(starting==1)):
    df.drop(df[df.person < starting].index, inplace= True)
    df.drop(df[df.following < starting].index, inplace= True)
  df.drop(df[df.person > Termination].index, inplace=True)
  df.drop(df[df.following > Termination].index, inplace=True)
  df.set_index(['person','following'], inplace=True)
  df.to_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+name+'.csv')
  del df
  gc.collect()
  df=pd.DataFrame()

In [0]:
trimCSV(1,2000,'edges')
df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/'+'edges.csv',names=['person','following'])
con=sqlite3.connect('tempSql.sql')
df.to_sql('Data',con)
cur=con.cursor()
del df
gc.collect()
for p in range(1,2001):
  for f in range(p,2001):
    cur.execute('SELECT COUNT(*) from Data WHERE person=? and following=? and EXISTS (SELECT * from Data WHERE person=? and following=?)',(p,f,f,p))
    for x in cur:
      if(x==(1,)):
        cur.execute('DELETE from Data WHERE person=? and following=?',(f,p))
      else:
        cur.execute('DELETE from Data WHERE person=? and following=?',(p,f))
    con.commit()
query='SELECT * from Data '
data=pd.read_sql(query,con)
data.set_index('person',inplace=True)
data.to_csv('processedData.csv',columns=['following'])
cur.close()

In [0]:
reqDataEndPoints=[200,400,600,800,1000,1200,1400,1600,1800,2000]
for size in reqDataEndPoints:
  df=pd.read_csv('/content/gdrive/My Drive/intern_iitropar/Processed Data of Twitter/processedData.csv')
  con=sqlite3.connect(outputPath+'tempSQl'+str(size)+'.sql')
  df.to_sql('Data',con)
  cur=con.cursor()
  cur.execute('DELETE from Data where person>? or following>? or person<? or following<?',(size,size,size-199,size-199))
  con.commit()
  del df
  data=pd.read_sql('SELECT * from Data',con)
  data.set_index('person',inplace=True)
  data.to_csv(outputPath+'processedData'+str(size)+'.csv',columns=['following'])
  del data
  gc.collect()